In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.dpi'] = 70 #display 70 dpi in Jupyter Notebook, may consider100 dpi 
plt.rcParams['savefig.dpi'] = 300 #define 300 dpi for saving figures

import seaborn as sns
## here are some settings 
sns.set_style('whitegrid')
sns.set(rc={"figure.dpi":70, 'savefig.dpi':300}) #defining dpi setting
sns.set_context('notebook')
sns.set_style("ticks")
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina')
# Tells matplotlib to display images inline instead of a new window
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
random.seed(1000)

from time import time
import timeit #imports timeit module

C:\Users\LMS-Khatrib\AppData\Local\Temp\ipykernel_11832\641145125.py:13: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  set_matplotlib_formats('retina')


In [2]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.preprocessing import PolynomialFeatures

import sklearn.linear_model as skl_lm
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn import neighbors
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis 
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.tree import DecisionTreeRegressor 
from sklearn.tree import DecisionTreeClassifier 
from sklearn.tree import export_graphviz
from sklearn.svm import SVC

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import train_test_split

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import mean_squared_error

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [3]:
df2017 = pd.read_csv('JORN17_dataset_v2.csv', na_values = '?').dropna()
df2021 = pd.read_csv("SRER_2017_training_bi.csv", na_values = '?').dropna()

df2017 = df2017.reindex(columns=["OID_", "Id", "gridcode","Shape_Length", "Shape_Area","CH_mean", "ARVI_mean","ARVI_max","ARVI_med","EVI_mean","EVI_max","EVI_med","NDVI_mean","NDVI_max","NDVI_med","SAVI_mean","SAVI_max","SAVI_med", "Veg_class"])
df2021 = df2021.reindex(columns=["OID_", "Id", "gridcode","Shape_Length", "Shape_Area","CH_mean", "ARVI_mean","ARVI_max","ARVI_med","EVI_mean","EVI_max","EVI_med","NDVI_mean","NDVI_max","NDVI_med","SAVI_mean","SAVI_max","SAVI_med", "Veg_class"])

bigtest1_df = pd.read_csv('SRER17_pred.csv', na_values='?')
bg1 = bigtest1_df.drop(columns=["Veg_class"])

dffull2021 = bg1.dropna()
dffull2021 = dffull2021.reindex(columns = ["OID_", "Id", "gridcode","Shape_Length", "Shape_Area","CH_mean", "ARVI_mean","ARVI_max","ARVI_med","EVI_mean","EVI_max","EVI_med","NDVI_mean","NDVI_max","NDVI_med","SAVI_mean","SAVI_max","SAVI_med", "Veg_class"]).fillna(0)

In [4]:
df2017.Veg_class = df2017.Veg_class.map({'non-woody':0, 'woody':1})
df2021.Veg_class = df2021.Veg_class.map({'non-woody':0, 'woody':1})

# Predicting models (JORN 2017 -> SRER 2017)

In [5]:
crossvalidation = KFold(n_splits=5, random_state=2, shuffle=True)
model = DecisionTreeClassifier(max_depth=6)

#criterion='entropy'
#scores = cross_val_score(model, X, y, cv=crossvalidation)
#print('Accuracy of each fold: \n {}'.format(scores))
#print()
#print("Avg accuracy: {}".format(scores.mean()))

acc_score = [];
Truth = [];
Output = [];

start_time = 0
for train_index, test_index in crossvalidation.split(df2017): 
    
    X_train = df2017.iloc[train_index, 5:18]
    X_test = df2017.iloc[test_index, 5:18]
    Y_train = df2017.iloc[train_index, -1]
    Y_test = df2017.iloc[test_index, -1]

    model.fit(X_train, Y_train)
    pred_values = model.predict(X_test)

    acc = accuracy_score(Y_test, pred_values)
    acc_score.append(acc)

    Truth.extend(Y_test.values.reshape(Y_test.shape[0])) 
    Output.extend(pred_values)

elapsed = timeit.default_timer() - start_time



print("---Run time is %s seconds ---" % elapsed)
print()
print('Accuracy of each fold: \n {}'.format(acc_score))
print("Avg accuracy: {}".format(np.mean(acc_score)))
print('Std of accuracy : \n{}'.format(np.std(acc_score)))
print()
print(confusion_matrix(Truth, Output))
print()
print(classification_report(Truth, Output))

cm = confusion_matrix(Truth, Output)

sensitivity = cm[0][0]/(cm[0][0]+cm[0][1])
specificity = cm[1][1]/(cm[1][0]+cm[1][1])
precision = (cm[0][0])/(cm[0][0]+cm[1][0])
f1_score = (2*precision*sensitivity)/(precision+sensitivity)


print("Sensitivity: {}".format(specificity))
print("Specificity: {}".format(sensitivity))
print("precision: {}".format(precision))
print("f1_score: {}".format(f1_score))


---Run time is 7.463667 seconds ---

Accuracy of each fold: 
 [0.7660377358490567, 0.7811320754716982, 0.7509433962264151, 0.7377358490566037, 0.7504725897920604]
Avg accuracy: 0.7572643292791669
Std of accuracy : 
0.014927058858816143

[[ 615  362]
 [ 281 1391]]

              precision    recall  f1-score   support

           0       0.69      0.63      0.66       977
           1       0.79      0.83      0.81      1672

    accuracy                           0.76      2649
   macro avg       0.74      0.73      0.73      2649
weighted avg       0.75      0.76      0.75      2649

Sensitivity: 0.8319377990430622
Specificity: 0.6294779938587513
precision: 0.6863839285714286
f1_score: 0.6567004805125468


In [6]:
model = DecisionTreeClassifier(max_depth=6)
acc_score = [];
Truth = [];
Output = [];
testing = df2021.iloc[:,-1]
model.fit(df2017.iloc[:, 5:18], df2017.iloc[:,-1])
pred_values = model.predict(df2021.iloc[:, 5:18])

acc = accuracy_score(testing, pred_values)
acc_score.append(acc)
Truth.extend(testing.values.reshape(testing.shape[0])) 
Output.extend(pred_values)

cm = confusion_matrix(Truth, Output)

sensitivity = cm[0][0]/(cm[0][0]+cm[0][1])
specificity = cm[1][1]/(cm[1][0]+cm[1][1])
precision = (cm[0][0])/(cm[0][0]+cm[1][0])
f1_score = (2*precision*sensitivity)/(precision+sensitivity)


print("Sensitivity: {}".format(specificity))
print("Specificity: {}".format(sensitivity))
print("precision: {}".format(precision))
print("f1_score: {}".format(f1_score))
print(acc_score)

Sensitivity: 0.44247001332741004
Specificity: 0.5842911877394636
precision: 0.49292929292929294
f1_score: 0.5347359193513039
[0.5107167550126758]


In [7]:
random.seed(1000)
bigtest_df = dffull2021
finalPredicted = model.predict(bigtest_df.iloc[:,5:18])
bigtest_df["Veg_class"] = finalPredicted

In [8]:
woodyarea = 0
nonwoodyarea = 0 
totalarea = 0
index = -1
woody = 0
nw = 0
#print(bigtest_df.iat[index,4])
area = bigtest_df["Shape_Area"]
#print(area)

for i in bigtest_df["Veg_class"]:
    index += 1
    if i == 1:
        totalarea += bigtest_df.iat[index,4]
        woodyarea += bigtest_df.iat[index,4]
        #print(woodyarea)
        woody += 1
    if i == 0:
        totalarea += bigtest_df.iat[index,4]
        nonwoodyarea += bigtest_df.iat[index,4]
        #print(nonwoodyarea)
        nw += 1

FWCdt = woodyarea / totalarea * 100

In [9]:
#Bagging Model 

crossvalidation = KFold(n_splits=5, random_state=2, shuffle=True)
model = RandomForestClassifier(max_features = 10, random_state = 2)
#scores = cross_val_score(model, X, y, cv=crossvalidation)
#print('Accuracy of each fold: \n {}'.format(scores))
#print()
#print("Avg accuracy: {}".format(scores.mean()))
#print(model)


acc_score = [];
Truth = [];
Output = [];

for train_index, test_index in crossvalidation.split(df2017): 
    
    X_train = df2017.iloc[train_index, 5:18]
    X_test = df2017.iloc[test_index, 5:18]
    Y_train = df2017.iloc[train_index, -1]
    Y_test = df2017.iloc[test_index, -1]

    model.fit(X_train, Y_train)
    pred_values = model.predict(X_test)

    acc = accuracy_score(Y_test, pred_values)
    acc_score.append(acc)

    Truth.extend(Y_test.values.reshape(Y_test.shape[0])) 
    Output.extend(pred_values)

elapsed = timeit.default_timer() - start_time



print("---Run time is %s seconds ---" % elapsed)
print()
print('Accuracy of each fold: \n {}'.format(acc_score))
print("Avg accuracy: {}".format(np.mean(acc_score)))
print('Std of accuracy : \n{}'.format(np.std(acc_score)))
print()
print(confusion_matrix(Truth, Output))
print()
print(classification_report(Truth, Output))

cm = confusion_matrix(Truth, Output)

sensitivity = cm[0][0]/(cm[0][0]+cm[0][1])
specificity = cm[1][1]/(cm[1][0]+cm[1][1])
precision = (cm[0][0])/(cm[0][0]+cm[1][0])
f1_score = (2*precision*sensitivity)/(precision+sensitivity)

print("Sensitivity: {}".format(specificity))
print("Specificity: {}".format(sensitivity))
print("precision: {}".format(precision))
print("f1_score: {}".format(f1_score))

#statallfeatures_dic['Run Time'].append(elapsed)
#statallfeatures_dic['Accuracy'].append(np.mean(acc_score))
#statallfeatures_dic['Standard Error'].append(np.std(acc_score))
#statallfeatures_dic['Sensitivity'].append(sensitivity)
#statallfeatures_dic['Specificity'].append(specificity)
#statallfeatures_dic['Precision'].append(precision)
#statallfeatures_dic['F1_Score'].append(f1_score)

#indaccs_dic['LogReg']=acc_score

---Run time is 49.2327722 seconds ---

Accuracy of each fold: 
 [0.7622641509433963, 0.7943396226415095, 0.7716981132075472, 0.7396226415094339, 0.7844990548204159]
Avg accuracy: 0.7704847166244606
Std of accuracy : 
0.018904898454860507

[[ 621  356]
 [ 252 1420]]

              precision    recall  f1-score   support

           0       0.71      0.64      0.67       977
           1       0.80      0.85      0.82      1672

    accuracy                           0.77      2649
   macro avg       0.76      0.74      0.75      2649
weighted avg       0.77      0.77      0.77      2649

Sensitivity: 0.8492822966507177
Specificity: 0.6356192425793245
precision: 0.711340206185567
f1_score: 0.6713513513513514


In [10]:
model = RandomForestClassifier(max_features = 10, random_state = 2)
acc_score = [];
Truth = [];
Output = [];
testing = df2021.iloc[:,-1]
model.fit(df2017.iloc[:, 5:18], df2017.iloc[:,-1])
pred_values = model.predict(df2021.iloc[:, 5:18])

acc = accuracy_score(testing, pred_values)
acc_score.append(acc)
Truth.extend(testing.values.reshape(testing.shape[0])) 
Output.extend(pred_values)

cm = confusion_matrix(Truth, Output)

sensitivity = cm[0][0]/(cm[0][0]+cm[0][1])
specificity = cm[1][1]/(cm[1][0]+cm[1][1])
precision = (cm[0][0])/(cm[0][0]+cm[1][0])
f1_score = (2*precision*sensitivity)/(precision+sensitivity)


print("Sensitivity: {}".format(specificity))
print("Specificity: {}".format(sensitivity))
print("precision: {}".format(precision))
print("f1_score: {}".format(f1_score))
print(acc_score)

Sensitivity: 0.5117725455353176
Specificity: 0.5828544061302682
precision: 0.5254749568221071
f1_score: 0.552679382379655
[0.5459783360221249]


In [11]:
random.seed(1000)
bigtest_df = dffull2021
finalPredicted = model.predict(bigtest_df.iloc[:,5:18])
bigtest_df["Veg_class"] = finalPredicted

In [12]:
woodyarea = 0
nonwoodyarea = 0 
totalarea = 0
index = -1
woody = 0
nw = 0
#print(bigtest_df.iat[index,4])
area = bigtest_df["Shape_Area"]
#print(area)

for i in bigtest_df["Veg_class"]:
    index += 1
    if i == 1:
        totalarea += bigtest_df.iat[index,4]
        woodyarea += bigtest_df.iat[index,4]
        #print(woodyarea)
        woody += 1
    if i == 0:
        totalarea += bigtest_df.iat[index,4]
        nonwoodyarea += bigtest_df.iat[index,4]
        #print(nonwoodyarea)
        nw += 1

FWCbag = woodyarea / totalarea * 100

In [13]:
crossvalidation = KFold(n_splits=5, random_state=2, shuffle=True)
model = AdaBoostClassifier(n_estimators=500, learning_rate = 0.1, algorithm="SAMME.R", random_state=2)
#scores = cross_val_score(model, X, y, cv=crossvalidation)
#print('Accuracy of each fold: \n {}'.format(scores))
#print()
#print("Avg accuracy: {}".format(scores.mean()))
#scores = cross_val_score(model, X, y, cv=crossvalidation)
#print('Accuracy of each fold: \n {}'.format(scores))
#print()
#print("Avg accuracy: {}".format(scores.mean()))
#print(model)


acc_score = [];
Truth = [];
Output = [];

for train_index, test_index in crossvalidation.split(df2017): 
    
    X_train = df2017.iloc[train_index, 5:18]
    X_test = df2021.iloc[test_index, 5:18]
    Y_train = df2017.iloc[train_index, -1]
    Y_test = df2021.iloc[test_index, -1]

    model.fit(X_train, Y_train)
    pred_values = model.predict(X_test)

    acc = accuracy_score(Y_test, pred_values)
    acc_score.append(acc)

    Truth.extend(Y_test.values.reshape(Y_test.shape[0])) 
    Output.extend(pred_values)

elapsed = timeit.default_timer() - start_time



print("---Run time is %s seconds ---" % elapsed)
print()
print('Accuracy of each fold: \n {}'.format(acc_score))
print("Avg accuracy: {}".format(np.mean(acc_score)))
print('Std of accuracy : \n{}'.format(np.std(acc_score)))
print()
print(confusion_matrix(Truth, Output))
print()
print(classification_report(Truth, Output))

cm = confusion_matrix(Truth, Output)

sensitivity = cm[0][0]/(cm[0][0]+cm[0][1])
specificity = cm[1][1]/(cm[1][0]+cm[1][1])
precision = (cm[0][0])/(cm[0][0]+cm[1][0])
f1_score = (2*precision*sensitivity)/(precision+sensitivity)

print("Sensitivity: {}".format(specificity))
print("Specificity: {}".format(sensitivity))
print("precision: {}".format(precision))
print("f1_score: {}".format(f1_score))

#statallfeatures_dic['Run Time'].append(elapsed)
#statallfeatures_dic['Accuracy'].append(np.mean(acc_score))
#statallfeatures_dic['Standard Error'].append(np.std(acc_score))
#statallfeatures_dic['Sensitivity'].append(sensitivity)
#statallfeatures_dic['Specificity'].append(specificity)
#statallfeatures_dic['Precision'].append(precision)
#statallfeatures_dic['F1_Score'].append(f1_score)

#indaccs_dic['LogReg']=acc_score

---Run time is 107.2099883 seconds ---

Accuracy of each fold: 
 [0.5433962264150943, 0.5, 0.5245283018867924, 0.5811320754716981, 0.5916824196597353]
Avg accuracy: 0.548147804686664
Std of accuracy : 
0.034298379917433876

[[1034  319]
 [ 878  418]]

              precision    recall  f1-score   support

           0       0.54      0.76      0.63      1353
           1       0.57      0.32      0.41      1296

    accuracy                           0.55      2649
   macro avg       0.55      0.54      0.52      2649
weighted avg       0.55      0.55      0.52      2649

Sensitivity: 0.32253086419753085
Specificity: 0.7642276422764228
precision: 0.5407949790794979
f1_score: 0.6333843797856049


In [14]:
model = AdaBoostClassifier(n_estimators=500, learning_rate = 0.1, algorithm="SAMME.R", random_state=2)
acc_score = [];
Truth = [];
Output = [];
testing = df2021.iloc[:,-1]
model.fit(df2017.iloc[:, 5:18], df2017.iloc[:,-1])
pred_values = model.predict(df2021.iloc[:, 5:18])

acc = accuracy_score(testing, pred_values)
acc_score.append(acc)
Truth.extend(testing.values.reshape(testing.shape[0])) 
Output.extend(pred_values)

cm = confusion_matrix(Truth, Output)

sensitivity = cm[0][0]/(cm[0][0]+cm[0][1])
specificity = cm[1][1]/(cm[1][0]+cm[1][1])
precision = (cm[0][0])/(cm[0][0]+cm[1][0])
f1_score = (2*precision*sensitivity)/(precision+sensitivity)


print("Sensitivity: {}".format(specificity))
print("Specificity: {}".format(sensitivity))
print("precision: {}".format(precision))
print("f1_score: {}".format(f1_score))
print(acc_score)

Sensitivity: 0.42425588627276767
Specificity: 0.7576628352490421
precision: 0.5496872828353023
f1_score: 0.6371325010068465
[0.5846969347775985]


In [15]:
random.seed(1000)
bigtest_df = dffull2021
finalPredicted = model.predict(bigtest_df.iloc[:,5:18])
bigtest_df["Veg_class"] = finalPredicted

In [16]:
woodyarea = 0
nonwoodyarea = 0 
totalarea = 0
index = -1
woody = 0
nw = 0
#print(bigtest_df.iat[index,4])
area = bigtest_df["Shape_Area"]
#print(area)

for i in bigtest_df["Veg_class"]:
    index += 1
    if i == 1:
        totalarea += bigtest_df.iat[index,4]
        woodyarea += bigtest_df.iat[index,4]
        #print(woodyarea)
        woody += 1
    if i == 0:
        totalarea += bigtest_df.iat[index,4]
        nonwoodyarea += bigtest_df.iat[index,4]
        #print(nonwoodyarea)
        nw += 1

FWCada = woodyarea / totalarea * 100

In [17]:
crossvalidation = KFold(n_splits=5, random_state=2, shuffle=True)
model =  GradientBoostingClassifier(n_estimators = 500, 
                                           learning_rate = 0.1, 
                                           max_depth = 4, 
                                           random_state = 2)
#scores = cross_val_score(model, X, y, cv=crossvalidation)
#print('Accuracy of each fold: \n {}'.format(scores))
#print()
#print("Avg accuracy: {}".format(scores.mean()))

#scores = cross_val_score(model, X, y, cv=crossvalidation)
#print('Accuracy of each fold: \n {}'.format(scores))
#print()
#print("Avg accuracy: {}".format(scores.mean()))
#print(model)


acc_score = [];
Truth = [];
Output = [];

for train_index, test_index in crossvalidation.split(df2017): 
    
    X_train = df2017.iloc[train_index, 5:18]
    X_test = df2017.iloc[test_index, 5:18]
    Y_train = df2017.iloc[train_index, -1]
    Y_test = df2017.iloc[test_index, -1]

    model.fit(X_train, Y_train)
    pred_values = model.predict(X_test)

    acc = accuracy_score(Y_test, pred_values)
    acc_score.append(acc)

    Truth.extend(Y_test.values.reshape(Y_test.shape[0])) 
    Output.extend(pred_values)

elapsed = timeit.default_timer() - start_time



print("---Run time is %s seconds ---" % elapsed)
print()
print('Accuracy of each fold: \n {}'.format(acc_score))
print("Avg accuracy: {}".format(np.mean(acc_score)))
print('Std of accuracy : \n{}'.format(np.std(acc_score)))
print()
print(confusion_matrix(Truth, Output))
print()
print(classification_report(Truth, Output))

cm = confusion_matrix(Truth, Output)

sensitivity = cm[0][0]/(cm[0][0]+cm[0][1])
specificity = cm[1][1]/(cm[1][0]+cm[1][1])
precision = (cm[0][0])/(cm[0][0]+cm[1][0])
f1_score = (2*precision*sensitivity)/(precision+sensitivity)

print("Sensitivity: {}".format(specificity))
print("Specificity: {}".format(sensitivity))
print("precision: {}".format(precision))
print("f1_score: {}".format(f1_score))

#statallfeatures_dic['Run Time'].append(elapsed)
#statallfeatures_dic['Accuracy'].append(np.mean(acc_score))
#statallfeatures_dic['Standard Error'].append(np.std(acc_score))
#statallfeatures_dic['Sensitivity'].append(sensitivity)
#statallfeatures_dic['Specificity'].append(specificity)
#statallfeatures_dic['Precision'].append(precision)
#statallfeatures_dic['F1_Score'].append(f1_score)

#indaccs_dic['LogReg']=acc_score

---Run time is 276.6527788 seconds ---

Accuracy of each fold: 
 [0.7566037735849057, 0.7905660377358491, 0.7754716981132076, 0.7641509433962265, 0.7939508506616257]
Avg accuracy: 0.776148660698363
Std of accuracy : 
0.014499550608281621

[[ 635  342]
 [ 251 1421]]

              precision    recall  f1-score   support

           0       0.72      0.65      0.68       977
           1       0.81      0.85      0.83      1672

    accuracy                           0.78      2649
   macro avg       0.76      0.75      0.75      2649
weighted avg       0.77      0.78      0.77      2649

Sensitivity: 0.8498803827751196
Specificity: 0.6499488229273286
precision: 0.7167042889390519
f1_score: 0.6816961889425658


In [18]:
model =  GradientBoostingClassifier(n_estimators = 500, 
                                           learning_rate = 0.1, 
                                           max_depth = 4, 
                                           random_state = 2)
acc_score = [];
Truth = [];
Output = [];
testing = df2021.iloc[:,-1]
model.fit(df2017.iloc[:, 5:18], df2017.iloc[:,-1])
pred_values = model.predict(df2021.iloc[:, 5:18])

acc = accuracy_score(testing, pred_values)
acc_score.append(acc)
Truth.extend(testing.values.reshape(testing.shape[0])) 
Output.extend(pred_values)

cm = confusion_matrix(Truth, Output)

sensitivity = cm[0][0]/(cm[0][0]+cm[0][1])
specificity = cm[1][1]/(cm[1][0]+cm[1][1])
precision = (cm[0][0])/(cm[0][0]+cm[1][0])
f1_score = (2*precision*sensitivity)/(precision+sensitivity)


print("Sensitivity: {}".format(specificity))
print("Specificity: {}".format(sensitivity))
print("precision: {}".format(precision))
print("f1_score: {}".format(f1_score))
print("Accuracy: {}".format(acc_score))


Sensitivity: 0.5579742336739227
Specificity: 0.5617816091954023
precision: 0.5410516605166051
f1_score: 0.5512218045112782
Accuracy: [0.5598064070062226]


In [19]:
random.seed(1000)
bigtest_df = dffull2021
finalPredicted = model.predict(bigtest_df.iloc[:,5:18])
bigtest_df["Veg_class"] = finalPredicted

In [20]:
woodyarea = 0
nonwoodyarea = 0 
totalarea = 0
index = -1
woody = 0
nw = 0
#print(bigtest_df.iat[index,4])
area = bigtest_df["Shape_Area"]
#print(area)

for i in bigtest_df["Veg_class"]:
    index += 1
    if i == 1:
        totalarea += bigtest_df.iat[index,4]
        woodyarea += bigtest_df.iat[index,4]
        #print(woodyarea)
        woody += 1
    if i == 0:
        totalarea += bigtest_df.iat[index,4]
        nonwoodyarea += bigtest_df.iat[index,4]
        #print(nonwoodyarea)
        nw += 1

FWCgrad = woodyarea / totalarea * 100

In [21]:
print(FWCada)
print(FWCdt)
print(FWCbag)
print(FWCgrad)

86.2199732755323
40.84432868999988
45.62229219966271
43.53936885781361
